# Project 4: A Data Science Blog Post

As part of the submission for the 4th Project in the Udacity Data Science Nanodegree, this notebook code from the analysis of the Seattle Airbnb dataset available on Kaggle: https://www.kaggle.com/airbnb/seattle/data


In [296]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
!python -m pip install langdetect
from langdetect import detect



In [297]:
#read in review data
reviews = pd.read_csv(r"C:\Users\Adetomiwa\Desktop\DATA SCIENCE DEGREE\Python\Project 4\seattle\reviews.csv")

In [298]:
reviews.shape

(84849, 6)

In [299]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [300]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


In [301]:
# We'll drop the blank comment columns
reviews = reviews.dropna()

In [302]:
# Detect the languages of each comment for anomalies
comments = pd.DataFrame(reviews['comments'])
language = []
lang_index = []
errors = []
for index, row in comments.iterrows():
    try:
        language += [detect(str(comments.comments[index]))]
        lang_index += [index]
    except: 
        language += ['investigate']
        lang_index += [index]
        errors.append(index)
        print('this index gives an error:',index)

this index gives an error: 4300
this index gives an error: 8863
this index gives an error: 9112
this index gives an error: 10303
this index gives an error: 13890
this index gives an error: 28801
this index gives an error: 35051
this index gives an error: 51426
this index gives an error: 66132
this index gives an error: 67811
this index gives an error: 71206
this index gives an error: 77823


In [303]:
# Create a dataframe of the language values maintaining the original index of where they came from
lang_df = pd.DataFrame({'language':language}, index = lang_index)

In [304]:
# add, language to the dataframe, check how many languges to translate/investigate
reviews['language'] = lang_df
reviews['language'].value_counts()

en             83779
fr               239
de               211
zh-cn            160
es                79
ko                51
ro                42
nl                40
ja                29
pt                26
it                25
zh-tw             23
af                22
so                15
da                13
investigate       12
sv                10
ru                 8
no                 7
pl                 5
ca                 5
fi                 5
tl                 4
cy                 3
tr                 3
sl                 2
hr                 2
cs                 2
vi                 1
sw                 1
et                 1
el                 1
sk                 1
sq                 1
th                 1
fa                 1
hu                 1
Name: language, dtype: int64

In [305]:
reviews[reviews['language'] == 'investigate']

,listing_id,id,date,reviewer_id,reviewer_name,comments,language
4300,3953365,23777971,2014-12-09,20938014,Will,.,investigate
8863,7932696,46414097,2015-09-10,41768154,Tia,...,investigate
9112,2004931,38447903,2015-07-15,11978631,Hitesh,:),investigate
10303,1900698,20713118,2014-10-04,21526819,Chohye,-,investigate
13890,2274086,40490627,2015-07-31,29884964,Sarah,:),investigate
28801,4516960,30446205,2015-04-22,27856782,Kelly,*,investigate
35051,3706719,17646574,2014-08-15,9679616,Mika,.,investigate
51426,777159,13335174,2014-05-25,12937418,Maxime,.,investigate
66132,1048798,11382465,2014-03-31,13397363,Christine,..,investigate
67811,488268,12690425,2014-05-08,6192400,Jackie,.,investigate


- We see 2 comments with happy reviews and will keep these comments :) 
- Other comments we expect to drop off with further preprocessing and cleansing

In [306]:
# replace happy smileys with 'I am happy' and update their language to english
reviews.loc[reviews.comments == ":)", 'comments'] = '"I am happy"'
reviews.loc[(reviews.comments == '"I am happy"') & (reviews.language == 'investigate'), 'language'] = 'en'


In [307]:
# Drop other 'investigate' languages.
reviews = reviews[reviews.language != 'investigate']
reviews

,listing_id,id,date,reviewer_id,reviewer_name,comments,language
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...,en
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...,en
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb...",en
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...,en
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...,en
...,...,...,...,...,...,...,...
84844,3624990,50436321,2015-10-12,37419458,Ryan,The description and pictures of the apartment ...,en
84845,3624990,51024875,2015-10-17,6933252,Linda,We had an excellent stay. It was clean and com...,en
84846,3624990,51511988,2015-10-20,19543701,Jaime,"Gran ubicación, cerca de todo lo atractivo del...",es
84847,3624990,52814482,2015-11-02,24445024,Jørgen,"Very good apartement, clean and well sized. Si...",en


In [ ]:
from translate import Translator as tran
translator = tran(to_lang = 'en')
for idx, row in reviews.iterrows():
    if language != 'en':
        reviews.loc[idx, 'en_comments'] = translator.translate(row['comments'])

In [ ]:
reviews

In [ ]:
reviews.head()